In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.ndimage.filters import gaussian_filter

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
from ecephys_analyses.data import paths, load
from ecephys import utils
from ecephys.scoring import filter_ds, filter_by_cumulative_time_in_state

In [6]:
def ds2df(ds):
    s = ds.to_dataframe().stack()
    s.index.names = s.index.names[:-1] + ['region']
    return s.to_frame(name='power')

In [7]:
def smooth_spg(spg, smoothing_sigma):
    # width of the kernel is ~8x sigma. So sigma=1 -> ~8 frequency bins -> ~2Hz at 0.25Hz resolution. 
    df = spg.groupby(["frequency", "condition", "region"]).mean()
    df.loc[(slice(None), 'baseline', 'mpta_wm_ref'), :] = gaussian_filter(df.loc[(slice(None), 'baseline', 'mpta_wm_ref'), :], smoothing_sigma)
    df.loc[(slice(None), 'recovery', 'mpta_wm_ref'), :] = gaussian_filter(df.loc[(slice(None), 'recovery', 'mpta_wm_ref'), :], smoothing_sigma)
    df.loc[(slice(None), 'baseline', 'sr_wm_ref'), :] = gaussian_filter(df.loc[(slice(None), 'baseline', 'sr_wm_ref'), :], smoothing_sigma)
    df.loc[(slice(None), 'recovery', 'sr_wm_ref'), :] = gaussian_filter(df.loc[(slice(None), 'recovery', 'sr_wm_ref'), :], smoothing_sigma)
    return df

In [44]:
subject = "Allan"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]

In [45]:
bsl_spg = load.load_spectrogram(subject, experiment, bsl_condition)
bsl_hyp = load.load_hypnogram(subject, experiment, bsl_condition)
bsl_nrem_hyp = filter_by_cumulative_time_in_state(bsl_hyp, ["N1", "N2"], 2, 'h')
bsl_nrem_spg = filter_ds(bsl_spg, bsl_nrem_hyp, states).dropna(dim="time")
bsl_nrem_prm = bsl_nrem_spg / bsl_nrem_spg.mean(dim="time") * 100

In [46]:
rec_spg = load.load_spectrogram(subject, experiment, rec_condition)
rec_hyp = load.load_hypnogram(subject, experiment, rec_condition)
rec_nrem_hyp = filter_by_cumulative_time_in_state(rec_hyp, ["N1", "N2"], 2, 'h')
rec_nrem_spg = filter_ds(rec_spg, rec_hyp, states).dropna(dim="time")
rec_nrem_prm = rec_nrem_spg / bsl_nrem_spg.mean(dim="time") * 100

In [47]:
nrem_spg = pd.concat([ds2df(bsl_nrem_spg).assign(condition="baseline"), ds2df(rec_nrem_spg).assign(condition="recovery")])
nrem_prm = pd.concat([ds2df(bsl_nrem_prm).assign(condition="baseline"), ds2df(rec_nrem_prm).assign(condition="recovery")])

In [48]:
g = sns.relplot(data=smooth_spg(nrem_spg, 1), x='frequency', y='power', hue='condition', col='region', kind='line', aspect=(16/9), ci=None)
g.set(xscale='log', yscale='linear', ylabel='PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
g = sns.relplot(data=smooth_spg(nrem_spg, 1), x='frequency', y='power', hue='condition', col='region', kind='line', aspect=(16/9), ci=None)
g.set(xscale='log', yscale='log', ylabel='PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
g = sns.relplot(data=smooth_spg(nrem_prm, 1), x='frequency', y='power', hue='condition', col='region', kind='line', aspect=(16/9), ci=None)
g.set(xscale='log', ylabel='PSD (% BSL NREM)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …